In [1]:
!pip install weaviate openai weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.8/582.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 1.8 MB/s eta 0:00:00


In [2]:
import weaviate
#import os
#import requests
#import json
#import ast

In [3]:
from google.colab import userdata
WEAVIATE_URL= userdata.get('WEAVIATE_URL')
WEAVIATE_API_KEY = userdata.get('WEAVIATE_API_KEY')
print(WEAVIATE_URL)
print(WEAVIATE_API_KEY)

https://frjipbt6qtgn1jge3dq7w.c0.asia-southeast1.gcp.weaviate.cloud
ZmNLUXRFaU8zRFhUNm16Ql9lY2swSW1RbGVERks0N2dlbnZjblN5U0RTTHNpaUV4K29kWFhPcDJqemlRPV92MjAw


In [4]:
pip show weaviate-client

Name: weaviate-client
Version: 4.17.0
Summary: A python native Weaviate client
Home-page: https://github.com/weaviate/weaviate-python-client
Author: Weaviate
Author-email: hello@weaviate.io,
License: BSD 3-clause
Location: /usr/local/lib/python3.12/dist-packages
Requires: authlib, deprecation, grpcio, httpx, protobuf, pydantic, validators
Required-by: 


In [5]:
from weaviate.auth import Auth # Corrected import path
# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)

print(client.is_ready())

True


In [6]:
response = client.collections.list_all(simple=False)

print(response)

{'Movie': _CollectionConfig(name='Movie', description='Collection of movies with Azure OpenAI embeddings', generative_config=None, inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False, auto_tenant_creation=False, auto_tenant_activation=False), properties=[_Property(name='title', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer=None, vectorizer_configs={}), _Property(name='description', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tok

In [7]:
client.collections.delete_all()

In [8]:
response = client.collections.list_all(simple=False)

print(response)

{}


In [9]:
from weaviate.classes.config import Property, DataType, Configure

client.collections.create(
    name="Article",
    description="A collection of news articles for semantic search",
    vectorizer_config=Configure.Vectorizer.text2vec_openai(),  # Uses OpenAI embeddings
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="content", data_type=DataType.TEXT),
        Property(name="url", data_type=DataType.TEXT),
    ]
)
print("Collection 'Article' created successfully!")

Collection 'Article' created successfully!


In [10]:
response = client.collections.list_all(simple=False)

print(response)

{'Article': _CollectionConfig(name='Article', description='A collection of news articles for semantic search', generative_config=None, inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False, auto_tenant_creation=False, auto_tenant_activation=False), properties=[_Property(name='title', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=True), vectorizer='text2vec-openai', vectorizer_configs=None), _Property(name='content', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True,

In [11]:
# collection_name can be a string ("Article") or a list of strings (["Article", "Category"])
collection_name = "Article"

# THIS WILL DELETE THE SPECIFIED COLLECTION(S) AND THEIR OBJECTS
client.collections.delete(
    collection_name
)



In [ ]:
from weaviate import WeaviateClient
from weaviate.classes.config import Property, DataType, Configure


# --- Create collection using text2vec-weaviate and Snowflake model ---
client.collections.create(
    name="Movie",
    description="This collection contains all my data.",
    multi_tenancy_config=Configure.multi_tenancy(enabled=False),

    vectorizer_config=Configure.Vectorizer.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-l-v2.0",
        vectorize_collection_name=True  # includes collection name in embeddings
    ),



    properties=[
        Property(name="name", data_type=DataType.TEXT),
        Property(name="movie", data_type=DataType.TEXT),
        Property(name="description", data_type=DataType.TEXT),
    ],
)

print("Collection 'Movie' created successfully using text2vec-weaviate + Snowflake model!")

# --- Access the collection ---
collection = client.collections.get("Movie")

# --- Example insert data ---
movies = [
    {
        "name": "Christopher Nolan",
        "movie": "Inception",
        "description": "A skilled thief enters dreams to steal secrets and plant ideas."
    },
    {
        "name": "Denis Villeneuve",
        "movie": "Dune",
        "description": "A young nobleman becomes the leader of a desert planet’s people."
    },
]

for item in movies:
    collection.data.insert(properties=item)

print("Data inserted successfully using Weaviate’s internal Snowflake embedding model!")


✅ Collection 'Movie' created successfully using text2vec-weaviate + Snowflake model!
✅ Data inserted successfully using Weaviate’s internal Snowflake embedding model!


In [ ]:
# Example: Semantic search
query_text = "movies involving dreams and subconscious worlds"

response = collection.query.near_text(
    query=query_text,
    limit=3
)

for obj in response.objects:
    movie = obj.properties.get("movie")
    desc = obj.properties.get("description")
    distance = getattr(obj.metadata, "distance", None)

    print(f"Movie: {movie}")
    print(f"Description: {desc}")

    if distance is not None:
        print(f"🔹 Distance: {distance:.4f}")
    else:
        print("🔹 Distance: (not returned by server)")

    print("-----")


🎬 Movie: Inception
🧠 Description: A skilled thief enters dreams to steal secrets and plant ideas.
🔹 Distance: (not returned by server)
-----
🎬 Movie: Dune
🧠 Description: A young nobleman becomes the leader of a desert planet’s people.
🔹 Distance: (not returned by server)
-----


In [ ]:
client.collections.delete_all()

Schema Example
 {
  "classes": [
    {
      "class": "Article",
      "description": "A collection of news articles",
      "vectorizer": "text2vec-openai",
      "properties": [
        { "name": "title", "dataType": ["text"] },
        { "name": "content", "dataType": ["text"] },
        { "name": "url", "dataType": ["string"] }
      ]
    }
  ]
}


In [14]:
articles = [
    {"title": "AI in Healthcare", "content": "AI assists doctors in diagnosis.", "url": "https://example.com/ai-healthcare"},
    {"title": "Edge Computing Trends", "content": "Edge computing improves latency and privacy.", "url": "https://example.com/edge"},
    {"title": "Fog vs Cloud", "content": "Fog computing bridges cloud and edge devices.", "url": "https://example.com/fog"},
    {"title": "Machine Learning Basics", "content": "ML enables systems to learn from data.", "url": "https://example.com/ml"},
    {"title": "Natural Language Processing", "content": "NLP powers chatbots and language models.", "url": "https://example.com/nlp"},
    {"title": "Computer Vision", "content": "AI helps detect objects in real-time.", "url": "https://example.com/cv"},
    {"title": "AI Ethics", "content": "Ethical AI ensures fairness and transparency.", "url": "https://example.com/ethics"},
    {"title": "Generative AI", "content": "GenAI creates new text, images, and music.", "url": "https://example.com/genai"},
    {"title": "IoT and AI", "content": "AI enhances IoT data analysis and automation.", "url": "https://example.com/iot-ai"},
    {"title": "Autonomous Drones", "content": "AI enables drones to navigate and map terrain.", "url": "https://example.com/drones"}
]

In [15]:
from google.colab import userdata


AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
DEPLOYMENT_NAME = userdata.get('DEPLOYMENT_NAME')

In [16]:
client.collections.create(
    name="Article",
    vectorizer_config=weaviate.classes.config.Configure.Vectorizer.text2vec_openai(

        model="text-embedding-3-small"
    ),
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="content", data_type=DataType.TEXT),
        Property(name="author", data_type=DataType.TEXT),
    ]
)

In [17]:
collections = client.collections.list_all()
print(collections)

{'Article': _CollectionConfigSimple(name='Article', description=None, generative_config=None, properties=[_Property(name='title', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=True), vectorizer='text2vec-openai', vectorizer_configs=None), _Property(name='content', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=True), vectorizer='text2vec-openai', vectorizer_configs=None), _Property(name='author', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None,

In [18]:
from openai import AzureOpenAI

embedding_client = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version="2024-02-01"  # Confirm your API version in Azure portal
)

EMBED_MODEL = "text-embedding-3-small"


In [19]:
collection = client.collections.get("Article")

# Generate and insert
for article in articles:
    vector = embedding_client.embeddings.create(
        model="text-embedding-3-small",
        input=article["content"]
    ).data[0].embedding

    collection.data.insert(properties=article, vector=vector)

In [ ]:
query = "fight with courage and persistence"
query_vector = embedding_client.embeddings.create(
    model=EMBED_MODEL,
    input=query
).data[0].embedding

results = collection.query.near_vector(query_vector, limit=3)

print("\n Top matches:")
for obj in results.objects:
    print(f"- {obj.properties['title']}: {obj.properties['content'][:60]}...")


🔎 Top matches:
- AI Ethics: Ethical AI ensures fairness and transparency....
- AI in Healthcare: AI assists doctors in diagnosis....
- Computer Vision: AI helps detect objects in real-time....


In [22]:
from weaviate.classes.config import Property, DataType

articles = client.collections.use("Article")

articles.config.add_property(Property(name="onHomepage", data_type=DataType.BOOL))

In [23]:
articles = client.collections.use("Article")
articles_config = articles.config.get()

print(articles_config)

_CollectionConfig(name='Article', description=None, generative_config=None, inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False, auto_tenant_creation=False, auto_tenant_activation=False), properties=[_Property(name='title', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=True), vectorizer='text2vec-openai', vectorizer_configs=None), _Property(name='content', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_p

In [25]:
response = client.collections.list_all(simple=False)

print(response)

{'Article': _CollectionConfig(name='Article', description=None, generative_config=None, inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False, auto_tenant_creation=False, auto_tenant_activation=False), properties=[_Property(name='title', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=True), vectorizer='text2vec-openai', vectorizer_configs=None), _Property(name='content', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=Tr

In [ ]:
for col_name, col_config in response.items():
    print(f" Collection Name: {col_config.name}")
    print("Properties:")
    for prop in col_config.properties:
        print(f" - {prop.name} ({prop.data_type.value})")

📘 Collection Name: Article
Properties:
 - title (text)
 - content (text)
 - author (text)
 - url (text)
 - onHomepage (boolean)
📘 Collection Name: Movie
Properties:
 - name (text)
 - movie (text)
 - description (text)


In [27]:
from weaviate.classes.config import (
    Reconfigure,
    VectorFilterStrategy,
    ReplicationDeletionStrategy,
)

# Access the "Article" collection
articles = client.collections.use("Article")

# Optional: update collection metadata
articles.config.update(
    description="An updated collection description.",
    property_descriptions={
        "title": "The updated title description for article",
        "content": "The main content of the article",
        "author": "Author of the article",
        "url": "URL of the article",
        "onHomepage": "Whether this article appears on homepage"
    }

)


article_shards = articles.config.update_shards(
    status="READY"
)


print(article_shards)

# ============================================
# Display collection name and properties
# ============================================



{'tTWg4di3otPr': 'READY'}


In [29]:
import weaviate
from weaviate.classes.config import Property, DataType, Configure

# --- Connect to your hosted Weaviate instance ---
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,   # e.g., https://my-instance.weaviate.network
    auth_credentials=weaviate.auth.AuthApiKey(api_key=WEAVIATE_API_KEY)
)


In [ ]:
client.collections.create(
    name="Movie1",
    description="A collection of movies for semantic and filtered search.",

    vectorizer_config=Configure.Vectorizer.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-l-v2.0",
        vectorize_collection_name=True
    ),


    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="director", data_type=DataType.TEXT),
        Property(name="year", data_type=DataType.INT),
        Property(name="genre", data_type=DataType.TEXT),
        Property(name="description", data_type=DataType.TEXT),
    ],
)

print("'Movie' collection created successfully!")


✅ 'Movie' collection created successfully!


In [ ]:
collection = client.collections.get("Movie1")

movies = [
    {
        "title": "Inception",
        "director": "Christopher Nolan",
        "year": 2010,
        "genre": "Sci-Fi",
        "description": "A thief who steals secrets through dream-sharing technology."
    },
    {
        "title": "Interstellar",
        "director": "Christopher Nolan",
        "year": 2014,
        "genre": "Sci-Fi",
        "description": "A team travels through a wormhole to save humanity."
    },
    {
        "title": "The Dark Knight",
        "director": "Christopher Nolan",
        "year": 2008,
        "genre": "Action",
        "description": "Batman faces the Joker, a criminal mastermind."
    },
    {
        "title": "Dune",
        "director": "Denis Villeneuve",
        "year": 2021,
        "genre": "Adventure",
        "description": "A young nobleman becomes the leader of a desert planet’s people."
    }
]

for movie in movies:
    collection.data.insert(properties=movie)

print(" Sample movie data inserted!")


✅ Sample movie data inserted!


In [32]:
response = collection.query.fetch_objects(limit=10)
for obj in response.objects:
    print(obj.properties)


{'title': 'Interstellar', 'year': 2014, 'description': 'A team travels through a wormhole to save humanity.', 'director': 'Christopher Nolan', 'genre': 'Sci-Fi'}
{'title': 'The Dark Knight', 'year': 2008, 'description': 'Batman faces the Joker, a criminal mastermind.', 'director': 'Christopher Nolan', 'genre': 'Action'}
{'title': 'Dune', 'year': 2021, 'description': 'A young nobleman becomes the leader of a desert planet’s people.', 'director': 'Denis Villeneuve', 'genre': 'Adventure'}
{'title': 'Inception', 'year': 2010, 'description': 'A thief who steals secrets through dream-sharing technology.', 'director': 'Christopher Nolan', 'genre': 'Sci-Fi'}


In [ ]:
from weaviate.classes.query import Filter

# Find movies directed by Christopher Nolan after 2009
filter_condition = (
    Filter.by_property("director").equal("Christopher Nolan") &
    Filter.by_property("year").greater_than(2009)
)

response = collection.query.fetch_objects(
    filters=filter_condition,
    limit=5
)

for obj in response.objects:
    print(f" {obj.properties['title']} ({obj.properties['year']}) — {obj.properties['director']}")


🎬 Inception (2010) — Christopher Nolan
🎬 Interstellar (2014) — Christopher Nolan


In [ ]:
# Get one object to update
response = collection.query.fetch_objects(limit=1)
movie_id = response.objects[0].uuid

# Update description
collection.data.update(
    uuid=movie_id,
    properties={"description": "Updated: A thief enters dreams to alter reality itself."}
)

print(f" Updated movie ID: {movie_id}")


✅ Updated movie ID: 141089f2-2da7-4fcd-8c2a-777106ca0296


In [ ]:
# Delete a single object by UUID

collection.data.delete_by_id(uuid=movie_id)
print(f" Deleted object {movie_id}")

# Delete entire collection (careful!)
# client.collections.delete("Movie")
# print(" Collection 'Movie' deleted.")


🗑️ Deleted object 141089f2-2da7-4fcd-8c2a-777106ca0296


In [ ]:
query_text = "movies about dreams and alternate realities"

response = collection.query.near_text(
    query=query_text,
    limit=3
)

print(" Semantic Search Results:")
for obj in response.objects:
    movie = obj.properties.get("title")
    desc = obj.properties.get("description")
    distance = getattr(obj.metadata, "distance", None)
    print(f"- {movie}: {desc}")
    if distance is not None:
        print(f"  🔹 Similarity distance: {distance:.4f}")
    print("-----")


🎯 Semantic Search Results:
- Inception: A thief who steals secrets through dream-sharing technology.
-----
- Dune: A young nobleman becomes the leader of a desert planet’s people.
-----
- The Dark Knight: Batman faces the Joker, a criminal mastermind.
-----


In [ ]:
response = collection.query.bm25(
    query="dream technology",
    limit=3
)

print(" Keyword Search Results:")
for obj in response.objects:
    print(f"- {obj.properties['title']}: {obj.properties['description']}")


🔤 Keyword Search Results:
- Inception: A thief who steals secrets through dream-sharing technology.


In [ ]:
response = collection.query.hybrid(
    query="dream world and time travel",
    alpha=0.5,  # 0 = pure keyword, 1 = pure semantic
    limit=3
)

print(" Hybrid Search Results:")
for obj in response.objects:
    print(f"- {obj.properties['title']}: {obj.properties['description']}")


🌗 Hybrid Search Results:
- Inception: A thief who steals secrets through dream-sharing technology.
- Dune: A young nobleman becomes the leader of a desert planet’s people.
- The Dark Knight: Batman faces the Joker, a criminal mastermind.


In [43]:
!pip install llama-index llama-index-llms-azure-openai llama-index-embeddings-azure-openai llama-index-vector-stores-weaviate weaviate-client openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.3/191.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from google.colab import userdata

AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
DEPLOYMENT_NAME = userdata.get('DEPLOYMENT_NAME')

In [2]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core import Settings

In [3]:
llm = AzureOpenAI(
    model=DEPLOYMENT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version="2023-05-15",
)

In [4]:
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

# ====== Step 1: Initialize Azure OpenAI embedding model ======
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-small",  # model type
    deployment_name="text-embedding-3-small",
    api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version="2023-05-15",
)

In [5]:
Settings.llm = llm
Settings.embed_model = embed_model

In [6]:
from google.colab import userdata
WEAVIATE_URL= userdata.get('WEAVIATE_URL')
WEAVIATE_API_KEY = userdata.get('WEAVIATE_API_KEY')
print(WEAVIATE_URL)
print(WEAVIATE_API_KEY)

https://frjipbt6qtgn1jge3dq7w.c0.asia-southeast1.gcp.weaviate.cloud
ZmNLUXRFaU8zRFhUNm16Ql9lY2swSW1RbGVERks0N2dlbnZjblN5U0RTTHNpaUV4K29kWFhPcDJqemlRPV92MjAw


In [ ]:
# create_weaviate_collection_azure_embeddings.py
import os
import weaviate
from weaviate import auth
from openai import AzureOpenAI

# -----------------------------
#  Azure OpenAI Config
# -----------------------------

AZURE_OPENAI_EMBED_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBED_DEPLOYMENT", "text-embedding-3-small")
AZURE_OPENAI_API_VERSION = "2024-08-01-preview"

embedding_client = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
)

# -----------------------------
#  Weaviate Config
# -----------------------------

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=auth.AuthApiKey(api_key=WEAVIATE_API_KEY),
)

assert client.is_ready(), "❌ Weaviate not connected!"
print(" Connected to Weaviate")

# -----------------------------
#  Create a New Collection
# -----------------------------
COLLECTION_NAME = "Movie"



# Create schema manually (since embeddings come from Azure, not Weaviate)
client.collections.create(
    name=COLLECTION_NAME,
    description="Collection of movies with Azure OpenAI embeddings",
    vectorizer_config=None,  # we supply embeddings manually
    properties=[
        weaviate.classes.config.Property(name="title", data_type=weaviate.classes.config.DataType.TEXT),
        weaviate.classes.config.Property(name="description", data_type=weaviate.classes.config.DataType.TEXT),
        weaviate.classes.config.Property(name="director", data_type=weaviate.classes.config.DataType.TEXT),
    ],
)

print(f" Collection '{COLLECTION_NAME}' created successfully!")

collection = client.collections.get(COLLECTION_NAME)

# -----------------------------
#  Insert Data with Azure Embeddings
# -----------------------------
movies = [
    {
        "title": "Inception",
        "description": "A thief enters people's dreams to steal secrets and plant ideas.",
        "director": "Christopher Nolan",
    },
    {
        "title": "The Matrix",
        "description": "A hacker discovers reality is a simulation controlled by machines.",
        "director": "Lana Wachowski",
    },
    {
        "title": "Interstellar",
        "description": "Astronauts travel through a wormhole to find a new home for humanity.",
        "director": "Christopher Nolan",
    },
]

for movie in movies:
    text_for_embedding = f"{movie['title']} - {movie['description']}"
    embedding = (
        embedding_client.embeddings.create(
            model=AZURE_OPENAI_EMBED_DEPLOYMENT,
            input=text_for_embedding
        )
        .data[0]
        .embedding
    )
    collection.data.insert(properties=movie, vector=embedding)

print(" Sample movie data inserted successfully!")

# -----------------------------
#  Query by Similarity
# -----------------------------
query_text = "movie about exploring dreams and reality"
query_embedding = (
    embedding_client.embeddings.create(
        model=AZURE_OPENAI_EMBED_DEPLOYMENT,
        input=query_text
    )
    .data[0]
    .embedding
)

results = collection.query.near_vector(
    near_vector=query_embedding,
    limit=3,
)

print("\n Top Similar Movies:")
for obj in results.objects:
    title = obj.properties.get("title")
    desc = obj.properties.get("description")
    print(f"- {title}: {desc}")




print("\n Done.")


✅ Connected to Weaviate
✅ Collection 'Movie' created successfully!
✅ Sample movie data inserted successfully!

🎬 Top Similar Movies:
- Inception: A thief enters people's dreams to steal secrets and plant ideas.
- The Matrix: A hacker discovers reality is a simulation controlled by machines.
- Interstellar: Astronauts travel through a wormhole to find a new home for humanity.

✅ Done.


In [ ]:

from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.weaviate import WeaviateVectorStore


#  Wrap the Weaviate collection as a VectorStore
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=COLLECTION_NAME,
)

#  Create StorageContext (still valid)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# -----------------------------
#  Build LlamaIndex from the existing Weaviate collection
# -----------------------------
# Note: No 'service_context' parameter — use global Settings instead
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

print(" LlamaIndex connected to Weaviate collection successfully.")



✅ LlamaIndex connected to Weaviate collection successfully.


In [ ]:
# -----------------------------
#  Create RAG Query Engine
# -----------------------------
query_engine = index.as_query_engine(similarity_top_k=5)

# -----------------------------
#  Ask a Question (RAG)
# -----------------------------
question = "List the movies about artificial intelligence and their directors."
response = query_engine.query(question)

print("\n **RAG Response:**")
print(response)


🎬 **RAG Response:**
The movie about artificial intelligence is "The Matrix," directed by Lana Wachowski.


In [ ]:
# -----------------------------
#  Apply Metadata Filter (WHERE)
# -----------------------------
where_filter = {
    "path": ["director"],
    "operator": "Equal",
    "valueString": "Christopher Nolan",
}

# Create a retriever with filter
retriever = index.as_retriever(
    search_kwargs={
        "where": where_filter,
        "limit": 3
    }
)

In [13]:
# Wrap retriever into a query engine correctly
filtered_engine = index.as_query_engine(retriever=retriever)

filtered_question = "Summarize Nolan’s movies dealing with dreams or altered realities."
filtered_response = filtered_engine.query(filtered_question)

print("\n🎥 **Filtered RAG Response:**")
print(filtered_response)

TypeError: RetrieverQueryEngine.from_args() got multiple values for argument 'retriever'